# Classification Evaluator

In [2]:
from src.datasets import ClassDataset, aug_collate_fn, Augmentor
from functools import partial
from torch.utils.data import DataLoader
from src.learners import ClassifierLearner
from src.evaluators import ClassificationEvaluator
from src.util import Logger
import torch

device = 2
logger = Logger(1)
ds = ClassDataset("datasets/tea-grade-v2", "test")
aug = Augmentor([224, 224])
dl = DataLoader(ds, 16, collate_fn=partial(aug_collate_fn, aug=aug, aug_count=1))
ln = ClassifierLearner(
    enc_name="ResNet50",
    enc_params={"drop_out": 0.2},
    dec_params={"drop_out": 0.2, "use_batch_norm": True},
    devices=[device, device],
    logger=logger,
    head_trail=[34],
)
ev = ClassificationEvaluator(logger=logger, out_path="temp/vis", has_aug_ax=True)

with torch.no_grad():
    for i, batch in enumerate(dl):
        if i == 10:
            break
        out = ln(batch)
        ev.register(batch=batch, out=out)

report = ev.output()
assert type(report) == str